# キーフレーズ抽出アルゴリズムの比較
評判のいいSGRank, TextRank, SCAKE, RAKEを比べてみて、どれが配信の特徴を表現できているか確認する。

In [4]:
#python -m spacy download ja_core_news_md

In [5]:
import textacy

ja = textacy.load_spacy_lang("ja_core_news_md")

In [6]:
import pandas as pd

texts = []

datafolder = "../../../Data/Transcription_raw/"
def get_text(filename):
    df = pd.read_csv(filename)
    text = "。 ".join(df["text"].tolist())
    return text

example_files = [datafolder+"15-4.csv", datafolder+"2718-4.csv", datafolder+"983-4.csv", datafolder+"2084-4.csv"]

In [7]:
#https://qiita.com/python_walker/items/fbc8c3205d01129e6afc
def extract_phrases_spacy(text, method):
    doc = textacy.make_spacy_doc(text, lang=ja)
    keywords_with_score = [
        (kps, score) for kps, score in method(doc, normalize="lemma", topn=5)
    ]

    keywords = [keywords_with_score[i][0] for i in range(len(keywords_with_score))]
    scores = [keywords_with_score[i][1] for i in range(len(keywords_with_score))]

    return scores, keywords

from katodb.Analyzing.rake_ja import Tokenizer, JapaneseRake
def extract_phrases_Rake(text):
    tokenizer = Tokenizer(rawargs='-r "C:/Program Files/MeCab/etc/mecabrc" -u "C:/Program Files/MeCab/dic/unidic_kato.dic"')
    tokens = tokenizer.tokenize(text)
    
    rake = JapaneseRake()
    
    rake.extract_keywords_from_text(tokens)
    keywords = rake.get_ranked_phrases_with_scores()
    
    if len(keywords) > 0:
        return [x[0] for x in keywords], [x[1] for x in keywords]
    else:
        return [], []

In [8]:
from textacy.extract.keyterms import sgrank, textrank, scake
def compare(filename):
    text = get_text(filename)
    
    print("sgrank")
    print(extract_phrases_spacy(text, sgrank))
    
    print("textrank")
    print(extract_phrases_spacy(text, textrank))
    
    print("scake")
    print(extract_phrases_spacy(text, scake))
    
    print("rake")
    print(extract_phrases_Rake(text))

In [13]:
for file in example_files:
    print(file)
    print(get_text(file))
    print(compare(file))

Exception: Tokenization error: Input is too long, it can't be more than 49149 bytes, was 296077